In [154]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression

In [155]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [156]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [157]:
folder("models")

In [158]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [159]:
feature_list

{'SYN': ['Src Port',
  'Tot Bwd Pkts',
  'TotLen Fwd Pkts',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Mean',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow IAT Max',
  'Flow IAT Min',
  'Fwd IAT Min',
  'Bwd IAT Tot',
  'Bwd IAT Mean',
  'Bwd IAT Std',
  'Bwd IAT Max',
  'Bwd PSH Flags',
  'Fwd Header Len',
  'Fwd Pkts/s',
  'Bwd Pkts/s',
  'Pkt Len Mean',
  'Pkt Len Var',
  'FIN Flag Cnt',
  'SYN Flag Cnt',
  'ACK Flag Cnt',
  'Bwd Seg Size Avg',
  'Subflow Fwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Bwd Pkts',
  'Subflow Bwd Byts',
  'Fwd Act Data Pkts',
  'Active Max',
  'Active Min',
  'Idle Std',
  'Idle Min',
  'Label'],
 'HTTP': ['Dst Port',
  'Flow Duration',
  'Flow IAT Mean',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT Min',
  'Fwd Pkts/s',
  'FIN Flag Cnt',
  'Label'],
 'ACK': ['Flow IAT Mean', 'Label'],
 'UDP': ['Src Port', 'Flow IAT Std', 'Fwd IAT Std', 'Label'],
 'ARP': ['Src Port',
  'Dst Port',
  'Protocol',
  'TotLen Fwd Pkts',
  'T

In [160]:
file_list={
'SYN': ['../csvs\\dos-synflooding-1-dec.pcap_Flow.csv', '../csvs\\dos-synflooding-6-dec.pcap_Flow.csv'],
'HTTP': ['../csvs\\mirai-httpflooding-4-dec.pcap_Flow.csv', '../csvs\\http-flood -maze.csv'],
'ACK': ['../csvs\\mirai-ackflooding-4-dec.pcap_Flow.csv', '../csvs\\mirai-ackflooding-2-dec.pcap_Flow.csv'],
'UDP': ['../csvs\\mirai-udpflooding-4-dec.pcap_Flow.csv', '../csvs\\mirai-udpflooding-2-dec.pcap_Flow.csv'],
'ARP': ['../csvs\\mitm-arpspoofing-6-dec.pcap_Flow.csv', '../csvs\\mitm-arpspoofing-5-dec.pcap_Flow.csv'],
'SP': ['../csvs\\scan-hostport-3-dec.pcap_Flow.csv', '../csvs\\Edge-Port Scanning attack.pcap_Flow.csv'],
'BF': ['../csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv','../csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv'],
'OS': ['../csvs\\scan-portos-3-dec.pcap_Flow.csv', '../csvs\\BoT-IoT_OSScan.csv'],
'SCHD': ['../csvs\\scan-hostport-3-dec.pcap_Flow.csv', '../csvs\\scan-hostport-5-dec.pcap_Flow.csv'],
'MHDis': ['../csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv', '../csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv']}

In [161]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [162]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [169]:
import pandas as pd
import joblib
import os

# Đường dẫn tới tệp JSON
ATTACK_FEATURES_FILE = 'GA_output_ET.json'

# Hàm tải attack_features từ tệp JSON
def load_attack_features(json_file_path):
    with open(json_file_path, 'r') as file:
        attack_features = json.load(file)
    return attack_features

# Tải attack_features từ JSON
attack_features = load_attack_features(ATTACK_FEATURES_FILE)

model_paths =  {
    'SYN': './models/DT_SYN_model.pkl',
    'HTTP': './models/DT_HTTP_model.pkl',
    'ACK': './models/DT_ACK_model.pkl',
    'UDP': './models/DT_UDP_model.pkl',
    'ARP': './models/DT_ARP_model.pkl',
    'SP': './models/DT_SP_model.pkl',
    'BF': './models/DT_BF_model.pkl',
    'OS': './models/DT_OS_model.pkl',
    'SCHD': './models/DT_SCHD_model.pkl',
    'MHDis': './models/DT_MHDis_model.pkl'
}

# Hàm dự đoán kiểu tấn công
def predict_attack(input_data):
    results = []
    for attack, model_path in model_paths.items():
        # Tải mô hình và đặc trưng tương ứng
        model = joblib.load(model_path)
        features = attack_features[attack]
        
        # Lọc dữ liệu đầu vào theo đặc trưng
        filtered_data = input_data[features[:-1]]  # Bỏ cột 'Label'
        
        # Dự đoán
        predictions = model.predict(filtered_data)
        probabilities = model.predict_proba(filtered_data)[:, 1]
        
        # Lưu kết quả
        for pred, prob in zip(predictions, probabilities):
            result = {"Attack_Type": attack, "Prediction": int(pred), "Probability": float(prob)}

            if result not in results:
                results.append(result)
    
    return results

# Tổng hợp kết quả
def summarize_results(results):
    import pandas as pd
    results_df = pd.DataFrame(results)
    attack_predictions = results_df[results_df["Prediction"] == 1]
    
    if not attack_predictions.empty:
        # Trả về toàn bộ các kiểu tấn công được phát hiện
        attack_predictions = attack_predictions.sort_values(by="Probability", ascending=False)
        return attack_predictions.to_dict(orient="records")
    else:
        return []


# Chạy chương trình
if __name__ == "__main__":
    # Tải dữ liệu đầu vào để dự đoán
    test_data = pd.read_csv("./new_attack_data.csv")
    
    # Dự đoán
    results = predict_attack(test_data)
    
    # Tổng hợp kết quả
    final_results = summarize_results(results)
    
    if final_results:
        print("Detected Attacks:")
        for attack in final_results:
            print(f" - Attack Type: {attack['Attack_Type']}, Probability: {attack['Probability']}")
    else:
        print("No attacks detected.")


Detected Attacks:
 - Attack Type: SP, Probability: 1.0
 - Attack Type: OS, Probability: 1.0
 - Attack Type: SCHD, Probability: 1.0
 - Attack Type: MHDis, Probability: 1.0


In [167]:
import joblib  # for saving models

# Assume `results` is the DataFrame you are using to collect the training details

# Loop over all attack types and train a model for each
for attack_type in file_list:
    print(f"Training model for {attack_type} attack")

    # Load data for the current attack
    df_train = pd.read_csv(file_list[attack_type][0], usecols=feature_list[attack_type])
    X_train = df_train.iloc[:, 0:-1]
    df_train['Label'] = df_train['Label'].astype('category')
    y_train = df_train['Label'].cat.codes

    df_test = pd.read_csv(file_list[attack_type][1], usecols=feature_list[attack_type])
    X_test = df_test.iloc[:, 0:-1]
    df_test['Label'] = df_test['Label'].astype('category')
    y_test = df_test['Label'].cat.codes

    X = np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)

    # Hyperparameter tuning and model training
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.linspace(1, 32, 32, endpoint=True).astype(int),
        'min_samples_split': sp_randint(2, 10),
        'max_features': sp_randint(1, X_train.shape[1])
    }

    second = time()
    f1 = []
    clf = DecisionTreeClassifier()

    # Fit the model and calculate the F1 score
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict = clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
    avg_f1 = sum(f1) / len(f1)

    print(f"Training completed for {attack_type} with average F1: {avg_f1}")

    # Save the trained model for each attack type
    model_filename = f"DT_{attack_type}_model.pkl"
    joblib.dump(clf, open(f'./models/{model_filename}', 'wb'))
    print(f"Model saved as {model_filename}")


Training model for SYN attack
Training completed for SYN with average F1: 0.9920774211121652
Model saved as DT_SYN_model.pkl
Training model for HTTP attack
Training completed for HTTP with average F1: 1.0
Model saved as DT_HTTP_model.pkl
Training model for ACK attack
Training completed for ACK with average F1: 0.9991569450604618
Model saved as DT_ACK_model.pkl
Training model for UDP attack
Training completed for UDP with average F1: 1.0
Model saved as DT_UDP_model.pkl
Training model for ARP attack
Training completed for ARP with average F1: 0.9610384671738221
Model saved as DT_ARP_model.pkl
Training model for SP attack
Training completed for SP with average F1: 1.0
Model saved as DT_SP_model.pkl
Training model for BF attack
Training completed for BF with average F1: 0.8216098452407635
Model saved as DT_BF_model.pkl
Training model for OS attack
Training completed for OS with average F1: 0.9169560480928106
Model saved as DT_OS_model.pkl
Training model for SCHD attack
Training completed f

# RandomizedSearchCV  DT

In [165]:
from scipy.stats import randint
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  

    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
       
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
        
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.linspace(1, 32, 32, endpoint=True).astype(int),
        'min_samples_split': randint(2, 10),
        'max_features': randint(1, X_train.shape[1] + 1)  # Adding +1 to ensure scale is positive
    }

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN


HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.991818137395952    2.83     24      


100%|██████████| 50/50 [02:36<00:00,  3.13s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.738    24      


100%|██████████| 50/50 [00:22<00:00,  2.26it/s]


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9827890906915157   0.291    24      


100%|██████████| 50/50 [00:10<00:00,  4.90it/s]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.969822440886506    0.725    24      


100%|██████████| 50/50 [00:19<00:00,  2.58it/s]

    criterion      max_depth    max_features    min_samples_split        F1          Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  -----------  ------  ----  --------
 0  entropy                5              19                    5  0.99421   0             3.079    14  SYN
 1  entropy                2               8                    2  1         0             0.419     2  HTTP
 2  gini                  32               1                    2  0.998314  0             0.311    23  ACK
 3  gini                   7               3                    3  1         0             0.223    37  UDP
 4  entropy               14              24                    2  0.96792   0.0060254     0.337    34  ARP
 5  gini                   2              30                    5  1         0             0.254    38  SP
 6  gini                  23               9                    3  0.823944  0.00666371    0.426    35  BF
 7  entropy          

# LR

In [166]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],  # Initially, allow all
    'C': loguniform(1e-5, 100)
}

    # Ensure compatibility between the solver and the penalty
    for solver in param_grid['solver']:
        if solver == 'newton-cg':
            param_grid['penalty'] = ['none', 'l2']  # Only 'none' and 'l2' are valid for 'newton-cg'
        elif solver == 'liblinear':
            param_grid['penalty'] = ['l1', 'l2']  # 'liblinear' supports both 'l1' and 'l2'
        else:
            param_grid['penalty'] = ['l2', 'none']  # Other solvers support 'l2' or 'none'
     
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second = time()
        a, b, clf = run_random_search(LogisticRegression(), param_grid, X, y)
        f1 = []
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict = clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
        f1_result = sum(f1) / len(f1)
        f1 = np.array(f1)
        stndtd = f1.std()
        temp = list(a.values())
        temp = temp + [f1_result, stndtd, round(time() - second, 3), i, j]
        lines.append(temp)

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
default                             0.9280132490952585   1.633    0       


100%|██████████| 10/10 [15:27<00:00, 92.75s/it]


HTTP
default                             0.3333333333333333   0.092    0       


100%|██████████| 10/10 [00:51<00:00,  5.15s/it]


ACK
default                             0.2909196067090804   0.021    0       


...

SCHD
default                             0.8978659439606658   0.169    0       


100%|██████████| 10/10 [01:04<00:00,  6.46s/it]


MHDis
default                             0.47672317687310195  0.176    0       


100%|██████████| 10/10 [01:04<00:00,  6.48s/it]

               C  penalty    Solver           F1          Std     Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  -------  ----  --------
 0   0.233258     l2         lbfgs      0.970374  0             46.761     0  SYN
 1   2.52723      l2         lbfgs      0.964028  0             21.53      1  SYN
 2   3.50388      l1         liblinear  0.969665  0            195.088     2  SYN
 3   1.9936e-05   l2         newton-cg  0.945397  0             74.021     3  SYN
 4   0.0517194    l2         lbfgs      0.957531  0            112.746     4  SYN
 5   0.261764     l2         newton-cg  0.953145  0            113.599     5  SYN
 6   1.1593       l1         liblinear  0.969665  0            227.932     6  SYN
 7   0.00232309   l2         lbfgs      0.964028  0             16.664     7  SYN
 8   0.858753     l2         lbfgs      0.957531  0            105.92      8  SYN
 9  44.1357       l2         lbfgs      0.924729  1.11022e-16   13.164     9  SYN
10   0.0